In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import splitfolders
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
#save augmented images in folder
def augment_data(image_path, save_dir):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (512, 512))
    
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
    )
    
    for i in range (0,6):
        if i==0:
            augmented_img = img 
        else:
            params = datagen.get_random_transform(img.shape)
            augmented_img = datagen.apply_transform(img, params)
            augmented_img = np.clip(augmented_img, 0, 255).astype(np.uint8)
        save_path = os.path.join(save_dir, f"augmented_{i}_{os.path.basename(image_path)}")
        cv2.imwrite(save_path, cv2.cvtColor(augmented_img, cv2.COLOR_RGB2BGR))

In [ ]:
#augmenting each directory
names = ['Chaar Narayan Temple', 'Chyasin Dega','Krishna Mandir', 'Shree Bhimsen Temple', 'Vishwanath Temple']
sides = ['East','West','North','South']
directory = 'Dataset'
save_directory = 'Augmented Dataset'
for name in names:
    augmented_base_dir = os.path.join(save_directory,name)
    os.makedirs(augmented_base_dir, exist_ok=True)
    for side in sides:
        os.makedirs(os.path.join(augmented_base_dir,side), exist_ok=True)
        path=os.path.join(directory,name,side)
        save_path=os.path.join(save_directory,name,side)
        [augment_data(os.path.join(path,img),save_path) for img in os.listdir(path)]


In [5]:
#Splitting the dataset
names = ['Chaar Narayan Temple', 'Chyasin Dega','Krishna Mandir', 'Shree Bhimsen Temple', 'Vishwanath Temple']
directory = "Augmented Dataset"
output_directory = "Dataset After Split"
for name in names: 
    input_folder = os.path.join(directory,name)
    splitfolders.ratio(
        input_folder,
        output=os.path.join(output_directory,name), 
        seed=42,
        ratio=(.7, .2, .1), 
        group_prefix=None
    ) 

In [ ]:
#Rearranging the split directory
import os
import shutil

names = ['Chaar Narayan Temple', 'Chyasin Dega', 'Krishna Mandir', 'Shree Bhimsen Temple', 'Vishwanath Temple']
sides = ['East', 'West', 'North', 'South']
directory = 'Dataset After Split'
save_directory = 'Split'
split = ['train', 'test', 'val']

for spl in split:
    spl_dir = os.path.join(save_directory, spl)
    os.makedirs(spl_dir, exist_ok=True)
    
    for name in names:
        name_dir = os.path.join(spl_dir, name)
        os.makedirs(name_dir, exist_ok=True)
        
        for side in sides:
            side_dir = os.path.join(name_dir, side)
            os.makedirs(side_dir, exist_ok=True)
            
            path = os.path.join(directory, name, spl, side)
            save_path = os.path.join(save_directory, spl, name, side)
            
            shutil.copytree(path, save_path, dirs_exist_ok=True)
